<a href="https://colab.research.google.com/github/Antonio24ch/Materia_apps_ciencia_de_datos/blob/main/limpieza_eda.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [ ]:
!pip install boto3
!pip install pickle
!pip install yaml
!pip install sodapy

     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 135.1/135.1 KB 985.0 kB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 10.5/10.5 MB 36.6 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 79.6/79.6 KB 4.7 MB/s eta 0:00:00
Looking in indexes: https://pypi.org/simple, https://us-python.pkg.dev/colab-wheels/public/simple/
ERROR: Could not find a version that satisfies the requirement pickle (from versions: none)
ERROR: No matching distribution found for pickle
Looking in indexes: https://pypi.org/simple, https://us-python.pkg.dev/colab-wheels/public/simple/
ERROR: Could not find a version that satisfies the requirement yaml (from versions: none)
ERROR: No matching distribution found for yaml
Looking in indexes: https://pypi.org/simple, https://us-python.pkg.dev/colab-wheels/public/simple/
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 62.8/62.8 KB 2.7 MB/s eta 0:00:00
  Attempting uninstall: requests
    Found existing installation: requests 2.27.1
    Uninstalling requ

In [ ]:
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import seaborn as sns
import yaml
import pickle
import boto3

from datetime import date
from sodapy import Socrata

In [ ]:
with open('credentials_.yaml', 'r') as f:
    x= yaml.safe_load(f)


cargar_datos_s3: Esta función recibe como parámetros el bucket de S3 y la ruta donde se encuentra el archivo con los datos de inspecciones que queremos revisar.           
Para saber qué archivo bajar tendrás que ocupar la función list_objects_v2 de boto3 e identificar el nombre del pkl que almacenaste.
 

In [ ]:
def cargar_datos_s3(bucket, bucket_path):
    session = boto3.Session(
        aws_access_key_id = x['s3']['aws_access_key_id'],
        aws_secret_access_key = x['s3']['aws_secret_access_key'],
        aws_session_token = x['s3']['aws_session_token']
    )

    s3 = session.resource('s3')
    
    obj = s3.Object(bucket, bucket_path).get()['Body'].read()
    data_loaded = pickle.loads(obj)
    
    return data_loaded

In [ ]:
session = boto3.Session(
    aws_access_key_id = x['s3']['aws_access_key_id'],
    aws_secret_access_key = x['s3']['aws_secret_access_key'],
    aws_session_token = x['s3']['aws_session_token']
)

s3 = session.client('s3')

bucket = "aplicaciones-cd-1" + x['iexe']['matricula']
key = "ingesta/inicial/"

In [ ]:
path = s3.list_objects_v2(Bucket=bucket, Prefix=key)['Contents'][0]['Key']

In [ ]:
inspections = cargar_datos_s3(bucket, path)

transformar_ingesta: Esta función recibe como parámetro los datos descargados desde el bucket de S3. 
Debe regresar un data frame que contiene los registros de las inspecciones históricas que se han realizado a  establecimientos de comida en Chicago.
 

In [ ]:
def transformar_ingesta(file_name):
     
    df = pd.DataFrame.from_dict(file_name)
    

    return df



In [ ]:
nuevo_df = transformar_ingesta(inspections)
nuevo_df.head(5)

,inspection_id,dba_name,aka_name,license_,facility_type,risk,address,city,state,zip,inspection_date,inspection_type,results,latitude,longitude,location,violations
0,2568436,SCHWAB REHAB HOSPITAL&CARE NET,SCHWAB REHAB HOSPITAL&CARE NET,2205586,Hospital,Risk 1 (High),1401 S CALIFORNIA AVE,CHICAGO,IL,60608,2022-12-13T00:00:00.000,Canvass,Pass,NaN,NaN,NaN,NaN
1,2567065,PARK MANOR,PARK MANOR ELEMENTARY,24841,School,Risk 1 (High),7037 S Rhodes (532E) AVE,CHICAGO,IL,60637,2022-11-15T00:00:00.000,Canvass,Pass,NaN,NaN,NaN,NaN
2,2564901,CHEEZE AND THANK YOU,CHEEZE AND THANK YOU,2872888,NaN,All,2046 E GRAND AVE,NaN,IL,NaN,2022-10-06T00:00:00.000,License,Not Ready,NaN,NaN,NaN,NaN
3,2561538,"PACINO'S RC, LLC",PACINOS,2683667,Restaurant,Risk 1 (High),1010 S DELANO CT,CHICAGO,IL,60605,2022-08-01T00:00:00.000,Canvass,No Entry,NaN,NaN,NaN,NaN
4,2555843,MC DONALD'S 6039,MC DONALD'S,2341076,Restaurant,Risk 2 (Medium),5656 W IRVING PARK RD,CHICAGO,IL,60634,2022-05-16T00:00:00.000,Canvass,Pass,41.95326311074034,-87.76931122260137,"{'latitude': '41.95326311074034', 'longitude':...",NaN



faltantes: Esta función imprime los valores faltantes de cada variable con la que contamos en el conjunto de datos de inspecciones.

In [ ]:
def valores_faltantes(df):
  
  faltantes = df.isnull().sum()

  return faltantes

In [ ]:
valores_faltantes(nuevo_df)

inspection_id        0
dba_name             0
aka_name             6
license_             0
facility_type       14
risk                 1
address              0
city                 3
state                0
zip                  2
inspection_date      0
inspection_type      0
results              0
latitude            17
longitude           17
location            17
violations         505
dtype: int64

elimina_faltantes_latitud_longitud: Esta función elimina registros que tienen valores nulos o vacíos en las columnas latitude o longitude. Recibe como parámetros la lista de columnas a eliminar por tener faltantes y el data frame al que pertenecen.

In [ ]:

def elimina_faltantes_latitud_longitud(df, columns):
    df = df.dropna(subset=columns)
    return df


In [ ]:
no_nulos_lat_long = elimina_faltantes_latitud_longitud(nuevo_df,['latitude','longitude'])
no_nulos_lat_long.isna().sum()

inspection_id        0
dba_name             0
aka_name             6
license_             0
facility_type       13
risk                 1
address              0
city                 2
state                0
zip                  1
inspection_date      0
inspection_type      0
results              0
latitude             0
longitude            0
location             0
violations         494
dtype: int64

imputar_faltantes: Esta función recibe como parámetros la columna que hay que imputar, el valor a imputar en caso de faltantes y el data frame al que pertenece la columna a imputar.
 
Debe regresar el data frame con los datos imputados.
 
Imputarás las columnas: license_,  zip,  state, facility_type, risk. La imputación se realizará utilizando los valores de la moda para cada una de las columnas.
 

In [ ]:
def imputar_valores(df, col):
    df[col].fillna(df[col].mode()[0], inplace=True)
    return df

In [ ]:
imputar_valores(nuevo_df, 'license_')
imputar_valores(nuevo_df, 'zip')
imputar_valores(nuevo_df, 'state')
imputar_valores(nuevo_df, 'facility_type')
imputar_valores(nuevo_df, 'risk')

,inspection_id,dba_name,aka_name,license_,facility_type,risk,address,city,state,zip,inspection_date,inspection_type,results,latitude,longitude,location,violations
0,2568436,SCHWAB REHAB HOSPITAL&CARE NET,SCHWAB REHAB HOSPITAL&CARE NET,2205586,Hospital,Risk 1 (High),1401 S CALIFORNIA AVE,CHICAGO,IL,60608,2022-12-13T00:00:00.000,Canvass,Pass,NaN,NaN,NaN,NaN
1,2567065,PARK MANOR,PARK MANOR ELEMENTARY,24841,School,Risk 1 (High),7037 S Rhodes (532E) AVE,CHICAGO,IL,60637,2022-11-15T00:00:00.000,Canvass,Pass,NaN,NaN,NaN,NaN
2,2564901,CHEEZE AND THANK YOU,CHEEZE AND THANK YOU,2872888,Restaurant,All,2046 E GRAND AVE,NaN,IL,60618,2022-10-06T00:00:00.000,License,Not Ready,NaN,NaN,NaN,NaN
3,2561538,"PACINO'S RC, LLC",PACINOS,2683667,Restaurant,Risk 1 (High),1010 S DELANO CT,CHICAGO,IL,60605,2022-08-01T00:00:00.000,Canvass,No Entry,NaN,NaN,NaN,NaN
4,2555843,MC DONALD'S 6039,MC DONALD'S,2341076,Restaurant,Risk 2 (Medium),5656 W IRVING PARK RD,CHICAGO,IL,60634,2022-05-16T00:00:00.000,Canvass,Pass,41.95326311074034,-87.76931122260137,"{'latitude': '41.95326311074034', 'longitude':...",NaN
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
995,2568538,CMM FOOD SERVICES,FOODA,2622051,Restaurant,Risk 1 (High),6301 S HALSTED ST,CHICAGO,IL,60621,2022-12-14T00:00:00.000,Canvass Re-Inspection,Fail,41.779661336898904,-87.64463773881644,"{'latitude': '41.779661336898904', 'longitude'...",59. PREVIOUS PRIORITY FOUNDATION VIOLATION COR...
996,2567637,CONN'S CAFE & CATERING INC,CONN'S CAFE & CATERING INC,33554,Restaurant,Risk 1 (High),1535 W DEVON AVE,CHICAGO,IL,60660,2022-11-28T00:00:00.000,Canvass,Out of Business,41.997985218648225,-87.66910712039837,"{'latitude': '41.997985218648225', 'longitude'...",NaN
997,2570186,"PATIO BEEF, INC",PATIO BEEF,2279587,Restaurant,Risk 1 (High),6022 N BROADWAY,CHICAGO,IL,60660,2023-01-25T00:00:00.000,Canvass,Pass,41.99164923315811,-87.66046438723393,"{'latitude': '41.99164923315811', 'longitude':...",NaN
998,2567528,Starbucks Coffee#48855,STARBUCKS COFFEE,2616204,Restaurant,Risk 2 (Medium),4155 N CICERO AVE,CHICAGO,IL,60641,2022-11-25T00:00:00.000,Canvass,Pass,41.9568024778031,-87.74725863483394,"{'latitude': '41.9568024778031', 'longitude': ...",49. NON-FOOD/FOOD CONTACT SURFACES CLEAN - Com...


In [ ]:
nuevo_df.isna().sum()

inspection_id        0
dba_name             0
aka_name             6
license_             0
facility_type        0
risk                 0
address              0
city                 3
state                0
zip                  0
inspection_date      0
inspection_type      0
results              0
latitude            17
longitude           17
location            17
violations         505
dtype: int64

transformar_enteros: Esta función recibe como parámetros una lista de variables que se requieren transformar a valores enteros y el data frame al que pertenecen.
 
Debe regresar el data frame con las columnas transformadas.
 
Las variables que debes transformar a valores enteros son: inspection_id.

In [ ]:
def transformar_enteros(df, col):
  df[col] = df[col].astype(int)
  return df

In [ ]:
enteros = transformar_enteros(nuevo_df,['inspection_id'])
enteros.dtypes

inspection_id       int64
dba_name           object
aka_name           object
license_           object
facility_type      object
risk               object
address            object
city               object
state              object
zip                object
inspection_date    object
inspection_type    object
results            object
latitude           object
longitude          object
location           object
violations         object
dtype: object

transformar_flotantes: Esta función recibe como parámetros una lista de variables que se requieren transformar a valores flotantes y el data frame al que pertenecen.
           
 Debe regresar el data frame con las columnas transformadas.
 
Las variables que debes transformar a valores flotantes son: latitud, longitud.

In [ ]:
def transformar_flotantes(df, cols):
  df[cols] = df[cols].astype(float)
  return df

flotantes = transformar_flotantes(nuevo_df,['longitude','latitude'])
flotantes.dtypes

inspection_id               int64
dba_name                   object
aka_name                   object
license_                   object
facility_type              object
risk                       object
address                    object
city                       object
state                      object
zip                        object
inspection_date    datetime64[ns]
inspection_type            object
results                    object
latitude                  float64
longitude                 float64
location                   object
violations                 object
dtype: object

transformar_fechas: Esta función recibe como parámetro una lista de variables que se requieren convertir a fechas.
 
Debe regresar el data frame con las columnas transformadas.
 
Las variables que debes transformar a fechas son: inspection_date.

In [ ]:
def transformar_fechas(df, col):
  df[col] =  df[col].astype('datetime64[ns]')
  return df

In [ ]:
fecha = transformar_fechas(nuevo_df,['inspection_date'])
fecha.dtypes

inspection_id               int64
dba_name                   object
aka_name                   object
license_                   object
facility_type              object
risk                       object
address                    object
city                       object
state                      object
zip                        object
inspection_date    datetime64[ns]
inspection_type            object
results                    object
latitude                   object
longitude                  object
location                   object
violations                 object
dtype: object

data_profiling_string: Esta función recibe como parámetros una lista de variables categóricas a las que se les hará un data profiling, y el data frame al que pertenecen.
 
El Data profiling a realizar para cada columna consiste en obtener el número de valores diferentes por cada variable, el porcentaje de valores únicos y la moda.
           
            Debe regresar un data frame con estas métricas por variable categórica.
 

In [ ]:
def data_profiling_string(df, cols):
    
    result = pd.DataFrame(columns=['Columna', 'Tipo', 'Valores Diferentes', '% Valores Únicos', 'Moda'])
    
    for col in cols:
        nunique = df[col].nunique()
        unique_percent = round(nunique / df[col].count(), 2) * 100
        mode = df[col].mode()[0]
        result = pd.concat([result, pd.DataFrame({'Columna': col, 'Tipo': df[col].dtype.name, 
                                                  'Valores Diferentes': nunique,
                                                  '% Valores Únicos': unique_percent, 'Moda': mode}, index=[0])])
    
    return result

In [ ]:
data_profiling_string(nuevo_df,['facility_type', 'state', 'city'])

,Columna,Tipo,Valores Diferentes,% Valores Únicos,Moda
0,facility_type,object,33,3.0,Restaurant
0,state,object,2,0.0,IL
0,city,object,5,1.0,CHICAGO



data_profiling_fechas: Esta función recibe como parámetro una lista de variables de fecha a las que se les obtendrán las siguientes estadísticas: Número de valores diferentes por cada variable, moda, fecha mínima, fecha máxima, número de años diferentes, número de días totales diferentes (utiliza la resta de datetimes de pandas).

Debe regresar un data frame con estas métricas por variable de fecha.

In [ ]:
def data_profiling_fechas(df, cols):
    
    result = pd.DataFrame(columns=['Columna', 'Tipo', 'Valores Diferentes', 'Fecha mínima', 'Fecha máxima', 'Moda','Días Diferentes', 'Años Diferentes'])
    
    for col in cols:
            nunique = df[col].nunique()
            minimo =df[col].min()
            maximo =df[col].max()
            mode = df[col].mode()[0]
            years_unicos = df[col].dt.year.nunique()
            dias_dif = df[col].max() - df[col].min()



            result = pd.concat([result, pd.DataFrame({'Columna': col, 'Tipo': df[col].dtype.name, 'Valores Diferentes': nunique,
                                    'Fecha mínima': minimo, 'Fecha máxima': maximo,'Moda': mode,'Días Diferentes':dias_dif , 'Años Diferentes':years_unicos}, index=[0])])
   
    return result

In [ ]:
data_profiling_fechas(nuevo_df,['inspection_date'])

,Columna,Tipo,Valores Diferentes,Fecha mínima,Fecha máxima,Moda,Días Diferentes,Años Diferentes
0,inspection_date,datetime64[ns],266,2020-11-05,2023-03-07,2023-01-13,852 days,4


guardar_datos_s3: Esta función recibe como parámetros el nombre de tu bucket aplicaciones-cd-1, la ruta donde guardarás los datos limpios limpieza y el data frame que ya tiene los datos con los tipos correctos.
 
El nombre del archivo debe ser datos-limpios-2020-01-01.pkl. Donde el año, mes y día corresponde al día en que se ejecuta el script y se debe obtener de manera dinámica a través de la función today del paquete datetime.time

 

In [ ]:
def guardar_consecutiva(bucket_name, bucket_path, df):
    session = boto3.Session(
        aws_access_key_id = x['s3']['aws_access_key_id'],
        aws_secret_access_key = x['s3']['aws_secret_access_key'],
        aws_session_token= x['s3']['aws_session_token']
    )

    s3 = session.resource('s3')

    s3.Object(bucket_name, bucket_path).put(Body=df)

    print("La ingesta ha sido guardada exitosamente en el bucket")



In [ ]:
data = pickle.dumps(nuevo_df)
bucket_name = "aplicaciones-cd-1" + x['iexe']['matricula']
ruta =  "ingesta/datos-limpios-" + str(date.today()) + ".pkl"
guardar_consecutiva(bucket_name, ruta, data)
